In [1]:
# import libraries and modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gmaps
import scipy.stats as scistats
import random
import requests
import json
from datetime import datetime
from config import api_key
import time

In [2]:
# Kristina: Module to generate list of movie IDs# Krist
def highest_revenue():
    year_df = pd.DataFrame()
    decade_df = pd.DataFrame()
    for year in range(1960,2010): 
        time.sleep(0.25)
        response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' 
                            +  api_key + '&primary_release_year=' + str(year) + '&sort_by=revenue.desc&language=en-US')

        #store parsed json response
        highest_revenue = response.json() 
        # highest_revenue
        highest_revenue_films = highest_revenue['results']
        data_df = pd.DataFrame(highest_revenue_films)

        year_df[str(year)] = data_df['id']

    for x in range(0,5):    
        decade_list = []
        for year in range(1960+(x*10),1970+(x*10)):
            decade_list.append(year_df[str(year)])
        flat_decade_list = [item for x in decade_list for item in x]
        decade_df[str(year-9)] = flat_decade_list
    return decade_df

In [5]:
# Evan: Module to get list of people from cast
def top2people(movies):
    
    peopleDict = {'1960':[],'1970':[],'1980':[],'1990':[],'2000':[]}

    # loop through each movie in each column
    for column in movies.iteritems():
        for movie in column[1]:
            #print(movie)
            # request credits list for movie
            cast_url = f'https://api.themoviedb.org/3/movie/{movie}/credits?api_key={api_key}'
            data = requests.get(cast_url).json()
            
            time.sleep(0.1)

            if data['cast']:
                # get list of cast IDs
                idList = {}
                for n,x in enumerate(data['cast']):
                    idList[n] = x['cast_id']

                # determine top 2 IDs
                idDF = pd.DataFrame.from_dict(idList, orient='index', columns=['cast_id'])
                idDF.sort_values('cast_id', inplace=True)
                idDF.reset_index(drop=False, inplace=True)

                # get person ID for top 2
                person1 = data['cast'][idDF.iloc[0]['index']]['id']
                if len(data['cast']) > 1:
                    person2 = data['cast'][idDF.iloc[1]['index']]['id']

                # check for uniqueness 
                if person1 not in peopleDict[column[0]]:
                    peopleDict[column[0]].append(person1)
                if len(data['cast']) > 1:
                    if person2 not in peopleDict[column[0]]:
                        peopleDict[column[0]].append(person2)
 # convert dictionary of people IDs to DataFrame
    peopleDF = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in peopleDict.items()]))
    return peopleDF

In [4]:
data.head()

NameError: name 'data' is not defined

In [6]:
movies = highest_revenue()

In [7]:
people = top2people(movies)


In [16]:
people.head(10)

,1960,1970,1980,1990,2000
0,2090,46597,2.0,723.0,500.0
1,3359,31070,3.0,3416.0,15336.0
2,7301,13784,20006.0,11510.0,934.0
3,7302,4299,20011.0,4517.0,73421.0
4,3151,862,707.0,1204.0,31.0
5,4090,9857,7171.0,1205.0,9994.0
6,3636,55636,3460.0,1269.0,2461.0
7,2639,827,9309.0,1581.0,61962.0
8,5676,57329,14414.0,1100.0,1981.0
9,5961,34759,2177.0,4430.0,206.0


In [26]:
# Aruna getting the actor name and birthplace with city and state and country

# url = f'https://api.themoviedb.org/3/person/{person}?api_key={api_key}'

In [12]:
#Aruna code
names = []
birthplace = {}
for column in people.head(20).iteritems():
    birthplace[column[0]] = []
    for x in range(column[1].count()):
        time.sleep(0.25)
        ids = column[1][x]
        #print (ids)
        #print(type(ids))
        if ids != 'nan':
            url = f'https://api.themoviedb.org/3/person/{ids}?api_key={api_key}'
            response = requests.get(url).json()
            #names.append(response['name'])
        
        #convert dictionary of people IDs to DataFrame
            if response['name'] != 'null':
                 birthplace[column[0]].append(response['name']) 
birthDF = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in birthplace.items()]))
birthDF.head()
    

,1960,1970,1980,1990,2000
0,Kirk Douglas,Ali MacGraw,Mark Hamill,Patrick Swayze,Tom Cruise
1,Laurence Olivier,Ryan O'Neal,Harrison Ford,Demi Moore,Dougray Scott
2,Anthony Perkins,Burt Lancaster,Christopher Reeve,Macaulay Culkin,Russell Crowe
3,Janet Leigh,Dean Martin,Margot Kidder,Joe Pesci,Joaquin Phoenix
4,Jack Lemmon,George C. Scott,Dan Aykroyd,Julia Roberts,Tom Hanks


In [13]:
birthDF.head(20)


,1960,1970,1980,1990,2000
0,Kirk Douglas,Ali MacGraw,Mark Hamill,Patrick Swayze,Tom Cruise
1,Laurence Olivier,Ryan O'Neal,Harrison Ford,Demi Moore,Dougray Scott
2,Anthony Perkins,Burt Lancaster,Christopher Reeve,Macaulay Culkin,Russell Crowe
3,Janet Leigh,Dean Martin,Margot Kidder,Joe Pesci,Joaquin Phoenix
4,Jack Lemmon,George C. Scott,Dan Aykroyd,Julia Roberts,Tom Hanks
5,Shirley MacLaine,Karl Malden,John Belushi,Richard Gere,Helen Hunt
6,Paul Newman,Donald Sutherland,Gene Wilder,Kevin Costner,Mel Gibson
7,Eva Marie Saint,Elliott Gould,Richard Pryor,Mary McDonnell,D. B. Sweeney
8,Marcello Mastroianni,Phil Harris,Kareem Abdul-Jabbar,Arnold Schwarzenegger,Alfre Woodard
9,Anita Ekberg,Sterling Holloway,Lloyd Bridges,Sharon Stone,Jim Carrey


In [15]:
birthDF.to_csv('Namesdataout.csv')



In [30]:
# for person in person_id:
    


In [31]:
#     print (response)
# print (names)
# print (birthplace)

In [32]:
# ids = ['nan', 'NaN', 1234]
# for x in ids:

#     if x is float or x is int:
#         print(x)

In [33]:
# people['1990'].count()